In [24]:
# Setup and Imports
"""
CareerForge AI - AI-Powered Career Acceleration System
===========================================================================

This notebook implements a multi-agent system that helps job seekers:
- Optimize resumes for ATS systems
- Prepare for interviews with personalized questions
- Negotiate salaries with market data
- Research companies intelligently

Author: Vaishnavi Saundankar
Course: Kaggle 5-Day Agents Course - Capstone Project
Track: Agents for Good (Education)
"""

%pip install -q google-genai google-adk

In [25]:
# Core imports
import os
import json
import re
import uuid
import inspect
import asyncio
import traceback
from datetime import datetime
from collections import Counter
from typing import Dict, List, Any, Optional

# ADK imports
from google.adk.agents import Agent, LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService, Session
from google.adk.memory import InMemoryMemoryService
from google.adk.tools import ToolContext

# Gemini content types
from google.genai import types

print("Imports loaded successfully.")

Imports loaded successfully.


In [26]:
# Load GOOGLE_API_KEY

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

if not GOOGLE_API_KEY:
    print("GOOGLE_API_KEY not found in environment.")
    GOOGLE_API_KEY = input("Enter your GOOGLE_API_KEY here: ").strip()

# Set environment variable
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

print("GOOGLE_API_KEY loaded:", bool(GOOGLE_API_KEY))


GOOGLE_API_KEY loaded: True


In [27]:
PRIMARY_MODEL = "gemini-2.0-flash-exp"

TECH_SKILLS = [
    "Python","Java","JavaScript","Go","SQL","AWS","Azure","GCP","Docker",
    "TensorFlow","PyTorch","Scikit-learn","Microservices","REST API",
    "Kubernetes","DevOps","Data Pipelines","Transformer","Embeddings"
]

ALL_SKILLS = list(dict.fromkeys(TECH_SKILLS))

In [28]:
# Custom Tools

from google.adk.tools import ToolContext
from typing import List, Dict, Any, Optional
from collections import Counter
from datetime import datetime
import re

async def parse_resume(
    resume_text: str,
    tool_context: Optional[ToolContext] = None
) -> Dict[str, Any]:
    resume_text = (resume_text or "")
    text_lower = resume_text.lower()

    found_skills = []
    for skill in ALL_SKILLS:
        if skill.lower() in text_lower:
            found_skills.append(skill)

    # overall experience
    experience_match = re.search(r'(\d+)[\+\s]*(?:years?|yrs)', resume_text, re.IGNORECASE)
    experience_years = int(experience_match.group(1)) if experience_match else 0

    sections = {
        "has_experience": bool(re.search(r'\b(experience|work|employment)\b', text_lower)),
        "has_education": bool(re.search(r'\b(education|degree|university|bachelor|master)\b', text_lower)),
        "has_skills": bool(re.search(r'\b(skills|technologies|technical)\b', text_lower)),
        "has_contact": bool(re.search(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}\b', resume_text))
    }

    words = re.findall(r'\b[a-zA-Z]{3,}\b', text_lower)
    stopwords = {'the','and','for','with','this','that','from','have','will','your','experience','project','projects','skills','technical'}
    keywords = [w for w in words if w not in stopwords]
    top_keywords = [k for k, _ in Counter(keywords).most_common(20)]

    relevant_years = 0
    # split into sentences
    ml_keywords = ["machine learning", "ml engineer", "ml", "ai", "deep learning", "nlp", "natural language", "transformer", "pytorch", "tensorflow", "langchain"]
    sentences = re.split(r'[.\n]+', resume_text)
    for s in sentences:
        s_low = s.lower()
        if any(k in s_low for k in ml_keywords):
            m = re.search(r'(\d+)[\+\s]*(?:years?|yrs)', s_low)
            if m:
                try:
                    yrs = int(m.group(1))
                    relevant_years = max(relevant_years, yrs)
                except Exception:
                    pass

    if relevant_years == 0:
        relevant_years = min(experience_years, 5)

    return {
        "skills": list(dict.fromkeys(found_skills)),
        "experience_years": experience_years,
        "relevant_experience_years": int(relevant_years),
        "sections": sections,
        "word_count": len(resume_text.split()),
        "top_keywords": top_keywords
    }


async def parse_job_description(
    job_text: str,
    tool_context: Optional[ToolContext] = None
) -> Dict[str, Any]:
    job_text = (job_text or "")
    text_lower = job_text.lower()

    required_skills = []
    preferred_skills = []

    for skill in ALL_SKILLS:
        if skill.lower() in text_lower:
            # capture context around the match
            context_matches = re.finditer(r'.{0,50}' + re.escape(skill.lower()) + r'.{0,50}', text_lower)
            is_required = False
            for match in context_matches:
                context = match.group(0)
                if any(kw in context for kw in ["required", "must", "essential"]):
                    is_required = True
                    break

            if is_required:
                required_skills.append(skill)
            else:
                preferred_skills.append(skill)

    experience_match = re.search(r'(\d+)[\+\-\s]*(?:years?|yrs)', job_text, re.IGNORECASE)
    experience_required = int(experience_match.group(1)) if experience_match else None

    words = re.findall(r'\b[a-zA-Z]{4,}\b', text_lower)
    stopwords = {'the', 'and', 'for', 'with', 'this', 'that', 'from', 'have', 'will', 'your'}
    keywords = [w for w in words if w not in stopwords]
    keyword_freq = Counter(keywords).most_common(20)

    return {
        "required_skills": list(dict.fromkeys(required_skills)),
        "preferred_skills": list(dict.fromkeys(preferred_skills)),
        "experience_required": experience_required,
        "top_keywords": [k for k, _ in keyword_freq],
        "word_count": len(job_text.split())
    }


# calculate_ats_score

async def calculate_ats_score(
    resume_data: Dict[str, Any],
    job_data: Dict[str, Any],
    tool_context: Optional[ToolContext] = None
) -> Dict[str, Any]:
    if isinstance(resume_data, str):
        text = resume_data.lower()
        resume_skills = set(s for s in ALL_SKILLS if s.lower() in text)
        experience_years = 0
        relevant_experience = 0
    else:
        resume_skills = set(s.lower() for s in resume_data.get("skills", []))
        experience_years = int(resume_data.get("experience_years", 0) or 0)
        relevant_experience = int(resume_data.get("relevant_experience_years", 0) or 0)

    if isinstance(job_data, str):
        job_text = job_data.lower()
        job_required = [s for s in ALL_SKILLS if s.lower() in job_text]
        job_keywords = re.findall(r'\b[a-zA-Z]{4,}\b', job_text)
    else:
        job_required = [s.lower() for s in job_data.get("required_skills", [])]
        job_keywords = [k.lower() for k in job_data.get("top_keywords", [])]

    required_skills = set(s.lower() for s in job_required)

    if required_skills:
        skills_match = len(resume_skills.intersection(required_skills)) / len(required_skills) * 100
    else:
        skills_match = 100.0

    resume_keywords = set(k.lower() for k in resume_data.get("top_keywords", [])[:30]) if isinstance(resume_data, dict) else set()
    job_keywords_set = set(job_keywords[:30]) if job_keywords else set()
    if job_keywords_set:
        keyword_match = len(resume_keywords.intersection(job_keywords_set)) / len(job_keywords_set) * 100
    else:
        keyword_match = 100.0

    sections = resume_data.get("sections", {}) if isinstance(resume_data, dict) else {}
    format_score = (sum(int(bool(v)) for v in sections.values()) / len(sections) * 100) if sections else 0.0

    # incorporate relevant experience into score: if job asks X years, penalize if relevant_experience < required
    experience_required = job_data.get("experience_required") if isinstance(job_data, dict) else None
    experience_score = 100.0
    if experience_required:
        if relevant_experience >= experience_required:
            experience_score = 100.0
        else:
            experience_score = max(0.0, (relevant_experience / experience_required) * 100)

    # overall weighting: skills 45, keywords 25, format 15, experience 15
    overall_score = (skills_match * 0.45 + keyword_match * 0.25 + format_score * 0.15 + experience_score * 0.15)

    missing_skills = list(required_skills - resume_skills)
    missing_keywords = list(job_keywords_set - resume_keywords)[:10]

    suggestions = []
    if skills_match < 70 and missing_skills:
        suggestions.append(f"Add or emphasize these skills: {', '.join(missing_skills[:8])}")
    if keyword_match < 70 and missing_keywords:
        suggestions.append(f"Incorporate keywords: {', '.join(missing_keywords[:8])}")
    if not sections.get("has_skills"):
        suggestions.append("Add a dedicated Skills section (concise bullet list).")
    if experience_required and relevant_experience < experience_required:
        suggestions.append(f"Add relevant coursework/projects to show experience close to required ({experience_required} yrs).")

    return {
        "overall_score": round(overall_score, 1),
        "skills_match": round(skills_match, 1),
        "keyword_match": round(keyword_match, 1),
        "format_score": round(format_score, 1),
        "experience_score": round(experience_score, 1),
        "missing_skills": missing_skills[:10],
        "missing_keywords": missing_keywords,
        "suggestions": suggestions,
        "pass_threshold": overall_score >= 85
    }

In [29]:
!pip install --quiet beautifulsoup4 requests

In [30]:
# Salary Negotiation

import requests
from bs4 import BeautifulSoup
from urllib.parse import quote_plus
import time
import random
import re
from typing import Dict, Any, Optional

# REALISTIC SALARY DATA (Updated for 2024-2025)

SALARY_BASE_MAP = {
    # Entry-level (0-2 years RELEVANT experience)
    "ai engineer_entry": 95000,
    "ml engineer_entry": 100000,
    "software engineer_entry": 85000,
    "data scientist_entry": 90000,
    "data engineer_entry": 88000,

    # Mid-level (3-5 years)
    "ai engineer_mid": 135000,
    "ml engineer_mid": 145000,
    "software engineer_mid": 120000,
    "data scientist_mid": 125000,
    "data engineer_mid": 115000,

    # Senior (6-10 years)
    "ai engineer_senior": 175000,
    "ml engineer_senior": 185000,
    "software engineer_senior": 160000,
    "data scientist_senior": 165000,
    "data engineer_senior": 155000,

    # Lead/Staff (10+ years)
    "ai engineer_lead": 220000,
    "ml engineer_lead": 235000,
    "software engineer_lead": 200000,
    "data scientist_lead": 210000,
}

# Location multipliers
LOCATION_MULTIPLIERS = {
    "san francisco": 1.25,
    "sf": 1.25,
    "bay area": 1.25,
    "new york": 1.22,
    "ny": 1.22,
    "nyc": 1.22,
    "seattle": 1.18,
    "boston": 1.15,
    "los angeles": 1.12,
    "la": 1.12,
    "austin": 1.10,
    "chicago": 1.08,
    "denver": 1.05,
    "remote": 0.95,
    "default": 1.0
}


def determine_experience_level(relevant_years: int, total_years: int, is_transition: bool) -> str:
    """
    Determine experience level considering career transitions

    Args:
        relevant_years: Years in target field
        total_years: Total work experience
        is_transition: Whether this is a career transition
    """
    if is_transition or relevant_years <= 1:
        return "entry"
    elif relevant_years <= 2:
        return "entry"
    elif relevant_years <= 5:
        return "mid"
    elif relevant_years <= 10:
        return "senior"
    else:
        return "lead"


def normalize_position(position: str) -> str:
    """Normalize job title to standard format"""
    pos_lower = position.lower()

    if "machine learning" in pos_lower or "ml engineer" in pos_lower:
        return "ml engineer"
    elif "ai engineer" in pos_lower or "artificial intelligence" in pos_lower:
        return "ai engineer"
    elif "data scientist" in pos_lower:
        return "data scientist"
    elif "data engineer" in pos_lower:
        return "data engineer"
    elif "software engineer" in pos_lower:
        return "software engineer"
    else:
        return "software engineer"  # Default


def get_location_multiplier(location: str) -> float:
    """Get location salary multiplier"""
    loc_lower = location.lower()

    for key, multiplier in LOCATION_MULTIPLIERS.items():
        if key in loc_lower:
            return multiplier

    return LOCATION_MULTIPLIERS["default"]


def calculate_base_salary(
    position: str,
    relevant_experience_years: int,
    total_experience_years: int,
    is_career_transition: bool
) -> int:
    """
    Calculate base salary before location adjustment

    Career transition logic:
    - If transitioning, use entry-level salary regardless of total experience
    - Give small bonus for transferable skills (5-10%)
    """
    normalized_pos = normalize_position(position)
    level = determine_experience_level(
        relevant_experience_years,
        total_experience_years,
        is_career_transition
    )

    # Get base salary for position + level
    key = f"{normalized_pos}_{level}"
    base_salary = SALARY_BASE_MAP.get(key, SALARY_BASE_MAP.get(f"{normalized_pos}_entry", 90000))

    # If career transition with significant prior experience, add small bonus
    if is_career_transition and total_experience_years >= 3:
        # 5-10% bonus for transferable skills
        transferable_bonus = min(total_experience_years * 0.02, 0.10)  # Max 10%
        base_salary = int(base_salary * (1 + transferable_bonus))

    return base_salary


async def salary_negotiation_analysis(
    position: str,
    location: str,
    experience_years: int,
    current_offer: Optional[int] = None,
    relevant_experience_years: Optional[int] = None,
    is_career_transition: bool = False,
    tool_context: Optional[object] = None
) -> Dict[str, Any]:
    """
    Realistic salary analysis with career transition support

    Args:
        position: Job title
        location: Job location
        experience_years: Total years of work experience
        current_offer: Current salary offer (if any)
        relevant_experience_years: Years in TARGET field (None = same as experience_years)
        is_career_transition: True if switching careers (e.g., CRM → AI)

    Returns:
        Realistic salary analysis with market data
    """

    # If no relevant experience specified, assume all experience is relevant
    if relevant_experience_years is None:
        relevant_experience_years = experience_years

    # Auto-detect career transition
    if relevant_experience_years < experience_years - 1:
        is_career_transition = True

    # Calculate base salary
    base_salary = calculate_base_salary(
        position=position,
        relevant_experience_years=relevant_experience_years,
        total_experience_years=experience_years,
        is_career_transition=is_career_transition
    )

    # Apply location multiplier
    loc_multiplier = get_location_multiplier(location)
    market_median = int(base_salary * loc_multiplier)

    # Calculate market range (±15%)
    market_range = (int(market_median * 0.85), int(market_median * 1.15))

    # Determine experience level for messaging
    level = determine_experience_level(
        relevant_experience_years,
        experience_years,
        is_career_transition
    )

    # Build data sources
    sources = [
        f"Industry data for {level}-level {normalize_position(position)}",
        f"Location adjustment: {loc_multiplier}x for {location}"
    ]

    if is_career_transition:
        sources.append(
            f"Career transition adjustment: {relevant_experience_years} years relevant exp "
            f"(from {experience_years} years total)"
        )

    # Levels.fyi
    levels_val = None
    try:
        # Only fetch for non-entry level to avoid skewing data
        if level != "entry":
            levels_val = await fetch_levelsfyi_safe(position, location)
            if levels_val and 80000 <= levels_val <= 400000:  # Sanity check
                # Blend conservatively
                market_median = int((0.70 * market_median) + (0.30 * levels_val))
                sources.append(f"Levels.fyi supplementary data: ${levels_val:,}")
    except Exception:
        pass

    # Recalculate range after blending
    market_range = (int(market_median * 0.85), int(market_median * 1.15))

    # Decision strategy
    if current_offer:
        offer_vs_median = current_offer / market_median

        if offer_vs_median < 0.90:
            # Offer is significantly below market
            strategy = "COUNTER_UP"
            target = min(int(market_median * 1.05), market_range[1])
            confidence = 0.85
            reasoning = (
                f"Current offer (${current_offer:,}) is {int((1 - offer_vs_median) * 100)}% "
                f"below market median (${market_median:,}). Strong case for negotiation."
            )

        elif offer_vs_median > 1.10:
            # Offer is above market
            strategy = "ACCEPT"
            target = current_offer
            confidence = 0.90
            reasoning = (
                f"Current offer (${current_offer:,}) is {int((offer_vs_median - 1) * 100)}% "
                f"above market median (${market_median:,}). Excellent offer."
            )

        else:
            # Offer is close to market
            strategy = "NEGOTIATE_SLIGHTLY"
            target = int((current_offer + market_median) / 2)
            confidence = 0.70
            reasoning = (
                f"Current offer (${current_offer:,}) is within market range "
                f"(${market_range[0]:,} - ${market_range[1]:,}). Room for modest negotiation."
            )

    else:
        strategy = "WAIT_FOR_OFFER"
        target = market_median
        confidence = 0.75
        reasoning = (
            f"Target salary: ${market_median:,} based on {level}-level {position} "
            f"in {location}"
        )
        if is_career_transition:
            reasoning += f" (career transition with {relevant_experience_years} years relevant experience)"

    # Build raw samples
    raw_samples = [
        {
            "source": "industry_base",
            "level": level,
            "base_salary": base_salary,
            "adjusted_salary": market_median
        }
    ]
    if levels_val:
        raw_samples.append({
            "source": "levels.fyi",
            "sample_comp": levels_val
        })

    return {
        "market_median": market_median,
        "market_range": market_range,
        "sources_used": sources,
        "recommended_strategy": strategy,
        "target_salary": target,
        "confidence": round(float(confidence), 2),
        "raw_samples": raw_samples,
        "experience_level": level,
        "is_career_transition": is_career_transition,
        "relevant_years": relevant_experience_years,
        "total_years": experience_years,
        "reasoning": reasoning
    }


async def fetch_levelsfyi_safe(position: str, location: str) -> Optional[int]:
    """
    Safe wrapper for Levels.fyi fetching - returns None on any error
    """
    try:
        return fetch_levelsfyi(position, location, max_attempts=1, pause=0.5)
    except Exception:
        return None


def fetch_levelsfyi(position: str, location: str, max_attempts: int = 1, pause: float = 0.5) -> Optional[int]:
    """
    Best-effort Levels.fyi scraper (simplified, less aggressive)
    """
    try:
        if not position:
            return None

        query = f'{position} salary {location} site:levels.fyi'
        search_url = f"https://www.google.com/search?q={quote_plus(query)}"
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36"
        }

        for attempt in range(max_attempts):
            r = requests.get(search_url, headers=headers, timeout=5)
            if r.status_code != 200:
                return None

            soup = BeautifulSoup(r.text, "html.parser")

            # Find levels.fyi link
            href = None
            for a in soup.find_all("a", href=True):
                if "levels.fyi" in a.get("href", ""):
                    href = a.get("href")
                    break

            if not href:
                return None

            # Unwrap Google redirect
            m = re.search(r"/url\?q=(https?://[^&]+)", href)
            if m:
                url = m.group(1)
            else:
                url = href

            # Fetch page
            time.sleep(pause)
            p = requests.get(url, headers=headers, timeout=5)
            if p.status_code != 200:
                return None

            html = p.text

            # Extract salary (first big number)
            m2 = re.search(r'\$\s?([\d]{2,3}(?:,\d{3})+)', html)
            if m2:
                return int(m2.group(1).replace(",", ""))

            return None

    except Exception:
        return None

In [31]:
# install packages
!pip install --quiet google-search-results beautifulsoup4 requests

In [32]:
# API key handling

SERPAPI_API_KEY = os.getenv("SERPAPI_API_KEY") or None
if not SERPAPI_API_KEY:
    try:
        from getpass import getpass
        SERPAPI_API_KEY = getpass("Enter SERPAPI_API_KEY (input hidden): ").strip()
    except Exception:
        SERPAPI_API_KEY = input("Enter SERPAPI_API_KEY: ").strip()

if not SERPAPI_API_KEY:
    raise RuntimeError("SERPAPI_API_KEY not provided. Set env var or enter key when prompted.")
else:
    os.environ["SERPAPI_API_KEY"] = SERPAPI_API_KEY
    print("SERPAPI_API_KEY loaded:", bool(SERPAPI_API_KEY))

SERPAPI_API_KEY loaded: True


In [62]:
# Custom Job Scraper tool

import os
import requests
import asyncio
import time
from datetime import datetime, timedelta
from typing import Dict, List, Any, Optional
from serpapi import GoogleSearch
import re
import json

def serpapi_google_jobs_search(
    query: str,
    location: str = "United States",
    hours: int = 168,
    num_results: int = 20,
    verbose: bool = False
) -> List[Dict[str, Any]]:
    """Search using SerpAPI's Google Jobs engine with fallbacks and debug.

    - Falls back to organic_results when jobs_results is empty.
    - Handles several URL/link field variants.
    - Optional verbose to print basic diagnostics.
    """
    params = {
        "engine": "google_jobs",
        "q": query,
        "location": location,
        "api_key": os.environ.get("SERPAPI_API_KEY"),
        "num": num_results,
    }

    try:
        search = GoogleSearch(params)
        res = search.get_dict()
        if verbose:
            print("SerpAPI top keys:", list(res.keys())[:12])
        if "error" in res:
            if verbose:
                print("SerpAPI Error:", res["error"])
            return []

        jobs_raw = res.get("jobs_results") or []
        if not jobs_raw:
            if verbose:
                print("No jobs_results - scanning organic_results fallback...")
            for r in (res.get("organic_results") or [])[:50]:
                title = r.get("title","") or ""
                snippet = r.get("snippet","") or ""
                link = r.get("link") or r.get("displayed_link") or ""
                if any(k in (title + snippet).lower() for k in ["job", "hiring", "position", "engineer", "developer", "analyst"]):
                    jobs_raw.append({
                        "title": title,
                        "company_name": r.get("source") or r.get("displayed_link") or "",
                        "location": location,
                        "share_url": link,
                        "description": snippet
                    })

        jobs = []
        seen_urls = set()
        for job in jobs_raw:
            url = (
                job.get("share_url") or job.get("url") or
                (job.get("apply_options", [{}])[0].get("link") if isinstance(job.get("apply_options"), list) else "") or
                job.get("link") or job.get("job_link") or job.get("apply_url") or ""
            )
            if not url:
                url = job.get("link") or job.get("apply_url") or ""

            posted_ago = (
                job.get("detected_extensions", {}).get("posted_at") or
                job.get("posted_time") or
                job.get("date_posted") or
                job.get("timestamp") or
                ""
            )
            hours_ago = parse_time_ago(posted_ago)

            salary = extract_salary(job)
            location_str = job.get("location") or job.get("job_location") or job.get("location_name") or location

            # skip exact duplicate links early
            if url and url in seen_urls:
                continue

            j = {
                "title": job.get("title") or job.get("job_title") or "Unknown Position",
                "company": job.get("company_name") or job.get("company") or job.get("hiringOrganization") or "Unknown Company",
                "location": location_str,
                "url": url,
                "posted_hours_ago": hours_ago or 0,
                "posted_time": posted_ago or "Recent",
                "salary_range": salary,
                "description": (job.get("description") or job.get("snippet") or "")[:800],
                "job_id": job.get("job_id") or job.get("id") or "",
                "extensions": job.get("extensions", []) or job.get("detected_extensions", [])
            }
            if url:
                seen_urls.add(url)
            jobs.append(j)

        if verbose:
            print(f"SerpAPI returned {len(jobs)} job(s) after parsing.")
        return jobs

    except Exception as e:
        if verbose:
            print("SerpAPI Exception:", e)
        return []


def parse_time_ago(time_str: str) -> Optional[int]:
    """Parse time strings into hours"""
    if not time_str:
        return None

    time_str = time_str.lower()

    if "hour" in time_str:
        match = re.search(r'(\d+)\s*hour', time_str)
        if match:
            return int(match.group(1))
    elif "day" in time_str:
        match = re.search(r'(\d+)\s*day', time_str)
        if match:
            return int(match.group(1)) * 24
    elif "minute" in time_str or "just now" in time_str or "today" in time_str:
        return 1
    elif "week" in time_str:
        match = re.search(r'(\d+)\s*week', time_str)
        if match:
            return int(match.group(1)) * 168
    elif "month" in time_str:
        return 720

    return 24


def extract_salary(job: Dict) -> str:
    """Extract salary from job data"""
    detected = job.get("detected_extensions", {})

    if "salary" in detected:
        return detected["salary"]

    extensions = job.get("extensions", [])
    for ext in extensions:
        ext_str = str(ext)
        if "$" in ext_str:
            if re.search(r'\$[\d,]+', ext_str):
                return ext_str

    highlights = job.get("job_highlights", [])
    for highlight in highlights:
        items = highlight.get("items", [])
        for item in items:
            if "$" in item and any(word in item.lower() for word in ["salary", "pay", "compensation", "range"]):
                match = re.search(r'\$[\d,]+\s*-?\s*\$?[\d,]*', item)
                if match:
                    return match.group(0)

    description = job.get("description", "")
    salary_match = re.search(r'\$[\d,]+(?:\s*-\s*\$?[\d,]+)?(?:\s*(?:per year|annually|/year|/yr))?', description)
    if salary_match:
        return salary_match.group(0)

    return "Not specified"


def build_job_query(position: str, experience_years: int, include_skills: bool = False) -> str:
    """
    Build optimized job search query

    IMPORTANT: Keep queries simple for better results
    """
    if experience_years <= 2:
        return f'{position} entry level'
    elif experience_years <= 5:
        return f'{position}'
    elif experience_years <= 8:
        return f'{position} senior'
    else:
        return f'{position} senior'


def deduplicate_jobs(jobs: List[Dict], verbose: bool = False) -> List[Dict]:
    """
    Remove duplicate job postings using job_id or url fingerprint first,
    fallback to title/company/location.
    """
    seen = set()
    unique_jobs = []

    for i, job in enumerate(jobs):
        job_id = (job.get("job_id") or "").strip()
        url = (job.get("url") or "").strip()

        # Create fingerprint
        if job_id:
            fingerprint = f"id|{job_id}"
        elif url:
            fingerprint = f"url|{url}"
        else:
            title = (job.get("title") or "").lower().strip()
            company = (job.get("company") or "").lower().strip()
            location = (job.get("location") or "").lower().strip()

            title = re.sub(r'\s+', ' ', title).strip()

            fingerprint = f"fc|{title}|{company}|{location}"

        if fingerprint in seen:
            if verbose:
                print(f"Skipping duplicate: {job.get('title')} at {job.get('company')}")
            continue

        seen.add(fingerprint)
        unique_jobs.append(job)

    return unique_jobs

def diversify_job_results(jobs: List[Dict], max_per_company: int = 2, verbose: bool = False) -> List[Dict]:
    """
    Ensure diversity by limiting jobs per company.
    """
    company_counts = {}
    diverse_jobs = []

    for job in jobs:
        company = (job.get("company") or "Unknown").lower().strip()
        count = company_counts.get(company, 0)

        if count < max_per_company:
            diverse_jobs.append(job)
            company_counts[company] = count + 1
        else:
            if verbose:
                print(f"Skipping {company} (already have {count} jobs)")


    return diverse_jobs


def calculate_match_score(job: Dict, skills: List[str], experience_years: int) -> int:
    """Calculate match score (0-100)"""
    score = 50

    if skills:
        job_text = (
            job.get("description", "") + " " +
            job.get("title", "") + " " +
            " ".join(job.get("extensions", []))
        ).lower()

        matched_skills = sum(1 for skill in skills if skill.lower() in job_text)
        skill_ratio = matched_skills / len(skills) if skills else 0
        score += int(skill_ratio * 40)

    location = job.get("location", "").lower()
    if "remote" in location or "anywhere" in location:
        score += 10
    elif "hybrid" in location:
        score += 5

    hours_ago = job.get("posted_hours_ago", 999)
    if hours_ago <= 48:
        score += 10
    elif hours_ago <= 168:
        score += 5

    return min(score, 100)


def generate_match_reason(job: Dict, skills: List[str], score: int) -> str:
    """Generate match reason"""
    reasons = []

    if score >= 80:
        reasons.append("Excellent match")
    elif score >= 65:
        reasons.append("Strong match")
    else:
        reasons.append("Good match")

    job_text = (job.get("description", "") + " " + job.get("title", "")).lower()
    matched_skills = [s for s in skills[:5] if s.lower() in job_text]

    if matched_skills:
        reasons.append(f"skills: {', '.join(matched_skills[:3])}")

    location = job.get("location", "")
    if "remote" in location.lower():
        reasons.append("Remote position")

    hours_ago = job.get("posted_hours_ago", 0)
    if hours_ago <= 48:
        reasons.append("Very recent posting")

    return " • ".join(reasons)

def scrape_recent_jobs_real_sync(
    position: str,
    skills: List[str],
    hours: int = 168,
    experience_years: int = 0,
    location: str = "United States",
    num: int = 20,
    verbose: bool = True
) -> Dict[str, Any]:
    """
    Synchronous job scraper with SIMPLIFIED query strategy
    """

    query = build_job_query(position, experience_years)

    # Fetch jobs from SerpAPI
    jobs = serpapi_google_jobs_search(
        query=query,
        location=location,
        hours=hours,
        num_results=num,
        verbose=verbose
    )

    initial_count = len(jobs)

    if verbose:
        print(f"\n Initial fetch: {initial_count} jobs")

    # If we STILL got 0 jobs, try even simpler
    if initial_count == 0:

        query = position
        jobs = serpapi_google_jobs_search(
            query=query,
            location=location,
            hours=hours,
            num_results=num,
            verbose=verbose
        )
        initial_count = len(jobs)

    jobs = deduplicate_jobs(jobs, verbose=verbose)
    after_dedup = len(jobs)

    # Calculate match scores
    for job in jobs:
        match_score = calculate_match_score(job, skills, experience_years)
        job["match_score"] = match_score
        job["match_reason"] = generate_match_reason(job, skills, match_score)

        if verbose:
            print(f"  {job.get('title')[:50]:50s} Score: {match_score}/100")

    # Sort by score
    jobs.sort(key=lambda x: x.get("match_score", 0), reverse=True)

    jobs = diversify_job_results(jobs, max_per_company=2, verbose=verbose)
    after_diversification = len(jobs)

    # Take top 5
    final_jobs = jobs[:5]

    if verbose:
        print(f"\n{'='*60}")
        print(f"FINAL RESULTS: {len(final_jobs)} jobs")
        print(f"{'='*60}")
        for i, job in enumerate(final_jobs, 1):
            print(f"{i}. {job.get('title')} - {job.get('company')}")
            print(f"   Score: {job.get('match_score')}/100")
            print(f"   Reason: {job.get('match_reason')}")
        print()

    return {
        "total_found": len(jobs),
        "jobs": final_jobs,
        "search_criteria": {
            "query": query,
            "location": location,
            "hours_filter": f"Last {hours} hours ({hours//24} days)",
            "experience_filter": f"{experience_years} years",
            "note": "Skills used for scoring, not filtering"
        },
        "debug_info": {
            "initial_results": initial_count,
            "after_deduplication": after_dedup,
            "after_diversification": after_diversification,
            "final_returned": len(final_jobs),
            "query_used": query
        }
    }


async def scrape_recent_jobs(
    position: str,
    skills: List[str],
    hours: int = 168,
    experience_years: int = 0,
    location: str = "United States",
    tool_context: Optional[object] = None
) -> Dict[str, Any]:
    """
    Async wrapper for job scraping
    """
    loop = asyncio.get_event_loop()
    return await loop.run_in_executor(
        None,
        scrape_recent_jobs_real_sync,
        position,
        skills,
        hours,
        experience_years,
        location,
        20,
        True
    )

In [48]:
# AGENT 1: RESUME TAILOR

class ResumeTailorAgent:
    def __init__(self):
        self.agent = LlmAgent(
            name="ResumeTailorAgent",
            description="Tailors resumes to match job descriptions without inventing facts",
            model=PRIMARY_MODEL,
            tools=[parse_resume, parse_job_description],
            instruction=(
                "You are an expert resume writer. IMPORTANT CONSTRAINTS:\n"
                "1) Do NOT invent or assert that the candidate worked at any company unless it already appears in the resume input. Never add new employers or dates.\n"
                "2) Produce suggested edits only. Mark any added/changed lines with the prefix '[SUGGESTED]'.\n"
                "3) Preserve the original Experience section verbatim unless the user explicitly asks to rewrite it.\n"
                "4) Use outputs from parse_resume and parse_job_description tools when available and never 'hallucinate' skills or employment.\n"
                "5) Return two sections: 'suggested_changes' (bullet list) and 'safe_draft' (the original resume with suggested sections appended but not altering existing employers).\n"
            )
        )

    async def tailor_resume(self, resume: str, job_description: str) -> Dict[str, Any]:
        session_service = InMemorySessionService()
        memory_service = InMemoryMemoryService()
        runner = Runner(app_name="careerforge", agent=self.agent,
                        session_service=session_service, memory_service=memory_service)
        user_id = "user_1"
        session_id = f"tailor_{uuid.uuid4().hex[:8]}"
        # ensure session exists
        try:
            await session_service.create_session(app_name="careerforge", user_id=user_id, session_id=session_id)
        except Exception:
            try:
                await session_service.get_session(app_name="careerforge", user_id=user_id, session_id=session_id)
            except Exception:
                pass

        resume_data = await parse_resume(resume)
        job_data = await parse_job_description(job_description)
        query = (
            f"Resume (DO NOT invent employment):\n{resume}\n\n"
            f"Resume parsed metadata (json):\n{json.dumps(resume_data, indent=2)}\n\n"
            f"Job description parsed metadata (json):\n{json.dumps(job_data, indent=2)}\n\n"
            "Task: Suggest concrete, prioritized edits (exact lines or bullets) to make the resume match the job. "
            "Mark added lines with [SUGGESTED]. Do NOT change existing 'Experience' company names or dates. "
            "Return JSON with keys: suggested_changes (list of strings), safe_draft (full resume text with suggested sections appended or marked)."
        )

        content = types.Content(role="user", parts=[types.Part(text=query)])
        result_text = ""
        async for event in runner.run_async(user_id=user_id, session_id=session_id, new_message=content):
            if event and event.content:
                for part in event.content.parts:
                    if hasattr(part, "text") and part.text:
                        result_text += part.text

        parsed = None
        try:
            parsed = json.loads(result_text)
        except Exception:
            parsed = {"raw": result_text}

        return {"tailored_resume": parsed, "resume_data": resume_data, "job_data": job_data}

In [49]:
# AGENT 2: ATS SCORER

class ATSScorerAgent:
    def __init__(self):
        self.agent = LlmAgent(
            name="ATSScorerAgent",
            description="Scores resume ATS compatibility",
            model=PRIMARY_MODEL,
            tools=[parse_resume, parse_job_description, calculate_ats_score],
            instruction="You are an ATS expert. Provide exact-line edits based on the ATS score."
        )

    async def score_and_improve(self, resume: str, job_description: str) -> Dict[str, Any]:
        """
        FIXED: Calls tools directly, then uses LLM for suggestions
        """
        # Step 1: Parse resume and job
        try:
            resume_data = await parse_resume(str(resume))
            job_data = await parse_job_description(str(job_description))

            # Step 2: Calculate score
            ats_result = await calculate_ats_score(resume_data, job_data)

            # If already passing, return early
            if ats_result.get("pass_threshold"):
                return {
                    "ats_analysis": ats_result,
                    "notes": "Score above 85% - minimal changes needed"
                }

            # Step 3: Get LLM suggestions
            session_service = InMemorySessionService()
            memory_service = InMemoryMemoryService()
            runner = Runner(
                app_name="careerforge",
                agent=self.agent,
                session_service=session_service,
                memory_service=memory_service
            )

            user_id = "user_1"
            session_id = f"ats_{uuid.uuid4().hex[:8]}"

            try:
                await session_service.create_session(
                    app_name="careerforge",
                    user_id=user_id,
                    session_id=session_id
                )
            except:
                pass

            prompt = f"""ATS Score Results:
{json.dumps(ats_result, indent=2)}

Original Resume:
{resume}

Provide 3-5 specific line edits to improve the score. Return JSON with:
- overall_score
- missing_skills
- recommended_changes (list of exact edits)"""

            content = types.Content(role="user", parts=[types.Part(text=prompt)])
            response_text = ""

            async for event in runner.run_async(user_id=user_id, session_id=session_id, new_message=content):
                if event and event.content:
                    for part in event.content.parts:
                        if hasattr(part, "text") and part.text:
                            response_text += part.text

            # Parse LLM response
            try:
                parsed = json.loads(response_text.strip())
                return {"ats_analysis": parsed, "raw_score": ats_result}
            except:
                return {"ats_analysis": response_text, "raw_score": ats_result}

        except Exception as e:
            return {"error": f"ATS Scorer failed: {str(e)}"}

In [58]:
# AGENT 3: JOB SCRAPER

class JobScraperAgent:
    def __init__(self):
        self.agent = LlmAgent(
            name="JobScraperAgent",
            description="Finds recently posted jobs from real job sites",
            model=PRIMARY_MODEL,
            tools=[scrape_recent_jobs, parse_resume],
            instruction=(
                "Primary job discovery is done via the scrape_recent_jobs tool. "
                "Use parse_resume to get candidate skills. "
                "Return top matches with match_score and match_reason. "
                "Do NOT invent job postings."
            )
        )

    async def find_jobs(
        self,
        resume: str,
        target_position: str,
        location: str = "United States",
        hours: int = 168,
        max_results: int = 5
    ) -> Dict[str, Any]:
        """
        Deterministic job finder that returns stable JSON
        """
        # 1. Parse resume to extract skills & experience
        try:
            resume_data = await parse_resume(resume)
            skills = resume_data.get("skills", []) or []
            rel_exp = int(resume_data.get("relevant_experience_years", 0) or 0)
            total_exp = int(resume_data.get("experience_years", 0) or 0)
        except Exception as e:
            print(f"Resume parsing failed: {e}")
            skills = []
            rel_exp = 0
            total_exp = 0

        # 2. Call scraper directly
        try:
            scraped = await scrape_recent_jobs(
                position=target_position,
                skills=skills,
                hours=hours,
                experience_years=rel_exp or total_exp,
                location=location
            )
        except Exception as e:
            print(f"Scraper failed: {e}")
            return {
                "top_matches": [],
                "error": f"scrape_recent_jobs failed: {type(e).__name__}: {e}"
            }

        # 3. Extract jobs from scraper response
        jobs = []
        if isinstance(scraped, dict):
            jobs = scraped.get("jobs") or scraped.get("job_matches") or scraped.get("top_matches") or []
        elif isinstance(scraped, list):
            jobs = scraped

        if not jobs:
            print(f"Scraper returned 0 jobs")
            return {
                "top_matches": [],
                "debug_info": {
                    "scraper_response": scraped,
                    "position": target_position,
                    "skills": skills,
                    "location": location
                }
            }

        # 4. Already deduplicated and scored by scraper
        for job in jobs:
            if "match_score" not in job:
                try:
                    job["match_score"] = calculate_match_score(job, skills, rel_exp)
                except Exception:
                    job["match_score"] = 50

            if "match_reason" not in job:
                try:
                    job["match_reason"] = generate_match_reason(job, skills, job.get("match_score", 50))
                except Exception:
                    job["match_reason"] = "Match based on job requirements"

        # 5. Sort by score and return top results
        jobs.sort(key=lambda x: x.get("match_score", 0), reverse=True)
        top_jobs = jobs[:max_results]

        print(f"Returning {len(top_jobs)} jobs (from {len(jobs)} total)")

        return {
            "top_matches": top_jobs,
            "total_found": len(jobs),
            "search_criteria": {
                "position": target_position,
                "location": location,
                "hours": hours,
                "skills_used": skills[:5],
                "experience_years": rel_exp or total_exp
            },
            "debug_info": {
                "scraper_returned": len(jobs),
                "final_returned": len(top_jobs),
                "skills_matched": skills[:10]
            }
        }

In [51]:
# AGENT 4: INTERVIEW INTELLIGENCE

class InterviewIntelligenceAgent:
    def __init__(self):
        self.agent = LlmAgent(
            name="InterviewIntelligenceAgent",
            description="Researches companies and generates interview questions",
            model=PRIMARY_MODEL,
            tools=[parse_job_description],
            instruction="""You prepare candidates for interviews.
1. Parse job description to extract core skills and responsibilities.
2. Generate 5 technical and 5 behavioral questions tailored to the role.
3. For each question provide why they ask it, key points to cover, and a short example/framework answer.
4. Summarize 3 company-specific talking points.
Return structured output with keys: technical_questions, behavioral_questions, company_insights, talking_points."""
        )

    async def prepare_interview(self, job_description: str, company_name: str) -> Dict[str, Any]:
        session_service = InMemorySessionService()
        memory_service = InMemoryMemoryService()

        runner = Runner(
            app_name="careerforge",
            agent=self.agent,
            session_service=session_service,
            memory_service=memory_service
        )

        user_id = "user_1"
        session_id = f"interview_{uuid.uuid4().hex[:8]}"

        try:
            session = await session_service.create_session(
                app_name="careerforge",
                user_id=user_id,
                session_id=session_id
            )
        except Exception:
            session = await session_service.get_session(
                app_name="careerforge",
                user_id=user_id,
                session_id=session_id
            )

        query = f"""Prepare interview materials for {company_name}.

JOB DESCRIPTION:
{job_description}

Instructions:
- Parse the job description to extract responsibilities and required skills.
- Generate 5 technical and 5 behavioral questions tailored to the role.
- For each question include: why asked, key points to include, and a short example answer.
- Provide 3 company-specific talking points.
Return structured output with keys: technical_questions, behavioral_questions, company_insights, talking_points."""
        content = types.Content(role="user", parts=[types.Part(text=query)])
        response_text = ""

        async for event in runner.run_async(user_id=user_id, session_id=session.id, new_message=content):
            if event and event.content:
                for part in event.content.parts:
                    if hasattr(part, "text") and part.text:
                        response_text += part.text

        return {"interview_prep": response_text}

In [52]:
# AGENT 5: SALARY NEGOTIATION

class SalaryNegotiationAgent:
    def __init__(self):
        self.agent = LlmAgent(
            name="SalaryNegotiationAgent",
            description="Creates realistic negotiation strategy for career transitions and standard roles",
            model=PRIMARY_MODEL,
            tools=[salary_negotiation_analysis, parse_job_description, parse_resume],
            instruction=(
                "You are a compensation negotiation expert specializing in career transitions. "
                "CRITICAL: Parse the resume to determine if this is a career transition (e.g., CRM → AI). "
                "Use relevant_experience_years (years in TARGET field) vs total_experience_years. "
                "Base ALL salary recommendations on the tool output - never invent numbers. "
                "Return JSON with: target_range, counter_offer, fallback_offer, walk_away_point, "
                "confidence, conversation_scripts, reasoning."
            )
        )

    async def create_strategy(
        self,
        job_description: str,
        company_name: str,
        current_offer: Optional[int] = None,
        experience_years: int = None,
        target_position: Optional[str] = None,
        resume: Optional[str] = None
    ) -> Dict[str, Any]:
        """
        Creates realistic salary negotiation strategy
        Handles career transitions properly
        """

        # 1. Parse resume to detect career transition
        parsed_resume = {}
        relevant_exp = 0
        total_exp = 0
        is_transition = False

        if resume:
            try:
                parsed_resume = await parse_resume(resume)

                if isinstance(parsed_resume, dict):
                    # Extract experience data
                    relevant_exp = int(parsed_resume.get("relevant_experience_years", 0))
                    total_exp = int(parsed_resume.get("experience_years", 0))

                    # Detect career transition
                    if relevant_exp < total_exp - 1:
                        is_transition = True

                    # Check for transition keywords in summary
                    summary = str(parsed_resume.get("summary", "")).lower()
                    if any(word in summary for word in ["transition", "seeking", "aspiring", "pivoting"]):
                        is_transition = True

            except Exception as e:
                print(f"Resume parsing error: {e}")

        # Use provided experience_years as fallback
        if total_exp == 0 and experience_years:
            total_exp = experience_years

        # 2. Parse job description
        job_data = {}
        try:
            job_data = await parse_job_description(job_description)
        except Exception as e:
            job_data = {"error": f"parse_job_description failed: {e}"}

        # 3. Extract position and location
        def extract_title_from_text(text: str) -> Optional[str]:
            if not text:
                return None
            m = re.search(
                r'\b(?:senior|lead|principal|staff|junior|associate)?\s*'
                r'(ai|ml|machine learning|data|software)\s*'
                r'(engineer|scientist|analyst)\b',
                text,
                flags=re.IGNORECASE
            )
            if m:
                return m.group(0).strip().title()
            return None

        def extract_location_from_text(text: str) -> Optional[str]:
            if not text:
                return None
            patterns = [
                r"San Francisco,?\s*CA", r"New York,?\s*NY", r"Seattle,?\s*WA",
                r"Chicago,?\s*IL", r"Boston,?\s*MA", r"Austin,?\s*TX",
                r"Remote", r"Hybrid"
            ]
            for p in patterns:
                m = re.search(p, text, flags=re.IGNORECASE)
                if m:
                    return m.group(0)
            return None

        position = extract_title_from_text(job_description) or target_position or "Software Engineer"
        location = extract_location_from_text(job_description) or "Remote"

        # Override with job_data if available
        if isinstance(job_data, dict):
            position = job_data.get("job_title") or position
            location = job_data.get("location") or location

        position = str(position).strip()
        location = str(location).strip()

        # 4. Call salary analysis tool with career transition parameters
        try:
            salary_analysis = await salary_negotiation_analysis(
                position=position,
                location=location,
                experience_years=total_exp,
                relevant_experience_years=relevant_exp,
                is_career_transition=is_transition,
                current_offer=current_offer
            )
        except Exception as e:
            salary_analysis = {"error": f"salary_negotiation_analysis failed: {e}"}

        # 5. Build prompt for LLM
        transition_note = ""
        if is_transition:
            transition_note = f"""
⚠️ CAREER TRANSITION DETECTED:
- Total experience: {total_exp} years
- Relevant experience in {position}: {relevant_exp} years
- Salary recommendation is for ENTRY-LEVEL {position} with transferable skills bonus
"""

        prompt_parts = [
            "You are a compensation negotiation expert. Use ONLY the salary analysis below.",
            f"Company: {company_name}",
            f"Position: {position}",
            f"Location: {location}",
            transition_note,
            f"Current offer: ${current_offer:,}" if current_offer else "Current offer: None",
            "\nSalary Analysis (from tool):",
            json.dumps(salary_analysis, indent=2),
            "\nTask: Create negotiation strategy JSON with:",
            "- target_range: string (e.g., '$95k-$110k')",
            "- counter_offer: int",
            "- fallback_offer: int",
            "- walk_away_point: int",
            "- confidence: float (0-1)",
            "- conversation_scripts: list of 3-4 short negotiation phrases",
            "- reasoning: 2-3 sentence explanation",
            "\nReturn ONLY valid JSON, no markdown."
        ]

        prompt = "\n\n".join(prompt_parts)

        # 6. Run LLM agent
        session_service = InMemorySessionService()
        memory_service = InMemoryMemoryService()
        runner = Runner(
            app_name="careerforge",
            agent=self.agent,
            session_service=session_service,
            memory_service=memory_service
        )

        user_id = "user_1"
        session_id = f"salary_{uuid.uuid4().hex[:8]}"

        try:
            await session_service.create_session(
                app_name="careerforge",
                user_id=user_id,
                session_id=session_id
            )
        except Exception:
            pass

        content = types.Content(role="user", parts=[types.Part(text=prompt)])
        response_text = ""

        try:
            async for event in runner.run_async(
                user_id=user_id,
                session_id=session_id,
                new_message=content
            ):
                if event and event.content:
                    for part in event.content.parts:
                        if hasattr(part, "text") and part.text:
                            response_text += part.text
        except Exception as e:
            return {
                "negotiation_strategy": {"error": f"LLM failed: {e}"},
                "analysis": salary_analysis,
                "job_data": job_data
            }

        # 7. Parse LLM response
        parsed = None
        try:
            clean = response_text.strip()
            clean = re.sub(r"^```(?:json)?\s*", "", clean)
            clean = re.sub(r"\s*```$", "", clean)
            parsed = json.loads(clean)
        except Exception:
            parsed = {"raw": response_text}

        return {
            "negotiation_strategy": parsed,
            "analysis": salary_analysis,
            "job_data": job_data,
            "career_transition_detected": is_transition,
            "experience_breakdown": {
                "total_years": total_exp,
                "relevant_years": relevant_exp
            }
        }

In [63]:
# SYSTEM ORCHESTRATOR

class CareerForgeSystem:
    def __init__(self):
        self.agent1 = ResumeTailorAgent()
        self.agent2 = ATSScorerAgent()
        self.agent3 = JobScraperAgent()
        self.agent4 = InterviewIntelligenceAgent()
        self.agent5 = SalaryNegotiationAgent()
        print("✅ CareerForge AI System initialized with 5 agents")

    async def run_full_workflow(
        self,
        resume: str,
        target_position: str,
        job_description: Optional[str] = None,
        company_name: Optional[str] = None
    ) -> Dict[str, Any]:
        results = {}

        print("\n🔍 Step 1/5: Searching for jobs...")
        try:
            jobs_result = await self.agent3.find_jobs(resume=resume, target_position=target_position)
            if "error" in jobs_result:
                results["job_search_error"] = jobs_result["error"]
            else:
                results["job_search"] = jobs_result.get("top_matches") or jobs_result.get("job_matches") or jobs_result.get("jobs") or jobs_result
        except Exception as e:
            results["job_search_error"] = f"{type(e).__name__}: {e}"

        if not job_description:
            return results

        print("📝 Step 2/5: Tailoring resume...")
        try:
            tailor_result = await self.agent1.tailor_resume(resume=resume, job_description=job_description)
            if "error" in tailor_result:
                results["tailored_resume_error"] = tailor_result["error"]
                tailored_resume_text = resume
            else:
                tailored_resume_text = tailor_result.get("tailored_resume", resume)
                results["tailored_resume"] = tailored_resume_text
        except Exception as e:
            results["tailored_resume_error"] = f"{type(e).__name__}: {e}"
            tailored_resume_text = resume

        print("🎯 Step 3/5: Scoring ATS compatibility...")
        try:
            ats_result = await self.agent2.score_and_improve(resume=tailored_resume_text, job_description=job_description)
            if "error" in ats_result:
                results["ats_analysis_error"] = ats_result["error"]
            else:
                results["ats_analysis"] = ats_result.get("ats_analysis", ats_result)
        except Exception as e:
            results["ats_analysis_error"] = f"{type(e).__name__}: {e}"

        if not company_name:
            return results

        print("💼 Step 4/5: Preparing interview materials...")
        try:
            interview_result = await self.agent4.prepare_interview(job_description=job_description, company_name=company_name)
            if "error" in interview_result:
                results["interview_prep_error"] = interview_result["error"]
            else:
                results["interview_prep"] = interview_result.get("interview_prep", interview_result)
        except Exception as e:
            results["interview_prep_error"] = f"{type(e).__name__}: {e}"

        print("💰 Step 5/5: Creating negotiation strategy...")
        try:
            salary_result = await self.agent5.create_strategy(
                job_description=job_description,
                company_name=company_name,
                current_offer=None,
                experience_years=5
            )
            if "error" in salary_result:
                results["salary_strategy_error"] = salary_result["error"]
            else:
                results["salary_strategy"] = salary_result.get("negotiation_strategy", salary_result)
        except Exception as e:
            results["salary_strategy_error"] = f"{type(e).__name__}: {e}"

        return results

In [67]:
# DEMO

SAMPLE_RESUME_BEFORE = """
Alex Johnson
San Francisco, CA | (555) 123-9876 | alex.johnson.ai@gmail.com
GitHub: github.com/alex-ml | LinkedIn: linkedin.com/in/alex-johnson-ml

Summary
AI/ML Engineer with strong experience designing, training, and deploying machine learning and LLM-based systems.
Hands-on background in deep learning, NLP, GenAI, RAG pipelines, and cloud-native deployments (AWS/GCP/Azure).
Experienced in full ML lifecycle including data engineering, model development, evaluation, and production deployment.

Technical Skills
Programming: Python, SQL, JavaScript
ML/DL: PyTorch, TensorFlow, scikit-learn, XGBoost, HuggingFace Transformers
NLP/GenAI: LLM Fine-tuning, Prompt Engineering, RAG, Vector Databases (FAISS, Pinecone)
Cloud: AWS (SageMaker, Lambda), GCP (Vertex AI), Azure ML
Tools: Docker, Kubernetes, Git, CI/CD, Airflow, MLflow
Data Engineering: Pandas, Spark, ETL Pipelines

Experience
Machine Learning Engineer
TechNova AI — San Francisco, CA
Jan 2023 – Present
– Designed and deployed LLM-powered chatbots and intelligent assistants using LangChain and custom RAG pipelines.
– Built classification and recommendation models using PyTorch/TensorFlow, improving accuracy by 18–25%.
– Created REST API microservices for inference with <120ms latency at scale.
– Implemented monitoring pipelines and ML observability dashboards reducing model drift incidents by 30%.
– Deployed ML systems using AWS SageMaker and Azure ML with automated CI/CD.

AI Engineer Intern
CloudWorks Intelligence — Seattle, WA
Jun 2022 – Dec 2022
– Implemented transformer-based NLP models for summarization and information extraction.
– Fine-tuned BERT/T5 models improving F1-score by 14% on domain-specific datasets.
– Deployed models using Docker + GCP Vertex AI and Cloud Run.
– Built automated ETL workflows using Airflow for training data pipelines.

Projects
Generative AI Knowledge Assistant (LLM + RAG)
– Built a RAG-based retrieval system using FAISS + LangChain achieving 87% retrieval accuracy.
– Integrated HuggingFace embeddings and a fine-tuned LLaMA model with prompt optimization.

Image Captioning System (CNN-LSTM + Attention)
– Designed a CNN-LSTM attention model on Flickr8k dataset achieving BLEU score of 0.53.

ML-Powered Customer Churn Prediction
– Developed feature engineering pipelines and trained XGBoost & ANN models achieving ROC-AUC of 0.91.

Education
M.S. in Computer Science
University of Washington — 2021–2023

Certifications
AWS Machine Learning Specialty
TensorFlow Developer Certificate
DeepLearning.AI NLP Specialization
"""

SAMPLE_JOB = """
San Francisco, California
$110K/yr – $120K/yr

Capgemini Engineering
AI Engineer

Responsibilities
Design, develop, and deploy AI/ML models and Generative AI applications for production use cases.
Build end-to-end AI systems including data pipelines, model training infrastructure, and inference APIs.
Develop intelligent applications using Large Language Models (AI agents, chatbots, RAG systems, automation workflows).
Integrate AI models into applications through REST APIs, microservices, and cloud-native architectures.
Deploy and manage models on cloud platforms (Azure, AWS, or GCP) with focus on scalability and reliability.
Collaborate with cross-functional teams to translate business requirements into technical solutions.
Write clean, well-documented code and participate in code reviews.
Stay current with the latest research and emerging trends in AI/ML and LLMs.

Required
Bachelor's degree in Computer Science, Engineering, AI, Data Science, or related field.
Strong coursework in machine learning, deep learning, algorithms, and data structures.
Proficiency in Python programming with core libraries (NumPy, pandas, Matplotlib).
Experience with ML frameworks: TensorFlow, PyTorch, or scikit-learn.
Understanding of transformer architectures, embeddings, and how LLMs work.
Familiarity with REST APIs, vector databases, and prompt engineering concepts.
Knowledge of cloud AI platforms (Azure, AWS, or GCP).
Strong problem-solving, communication, and teamwork skills.
Hands-on experience through internships, co-ops, academic projects, or personal portfolio.

Preferred
Experience with Large Language Models (GPT, Claude, Llama) and OpenAI APIs or Hugging Face Transformers.
Familiarity with LLM frameworks (LangChain, LlamaIndex, Semantic Kernel).
Familiarity with Agentic AI frameworks (LangGraph, Pydantic, Strands, ADK).
Experience building AI agents, multi-agent systems, or autonomous workflows.
Knowledge of RAG (Retrieval-Augmented Generation) systems and vector databases.
Production deployment experience on Azure ML, AWS SageMaker, or GCP Vertex AI.
Understanding of Git, Docker, CI/CD practices, and MLOps tools.
Experience with model fine-tuning or training from scratch.
GitHub portfolio with documented AI/ML projects.
Kaggle competitions, hackathons, or open-source contributions.
"""

In [68]:
async def run_demo():
    """
    Main demo function that orchestrates the complete CareerForge workflow.
    Returns dict with results from all 5 agents.
    """
    print("="*80)
    print("CAREERFORGE AI - COMPLETE DEMO")
    print("="*80)

    # Initialize system
    try:
        system = CareerForgeSystem()
    except Exception as e:
        print(f"ERROR constructing CareerForgeSystem: {e}")
        traceback.print_exc()
        return {"error": "CareerForgeSystem initialization failed", "trace": traceback.format_exc()}

    # Verify workflow method exists
    workflow_fn = getattr(system, "run_full_workflow", None)
    if workflow_fn is None:
        print("CareerForgeSystem has no 'run_full_workflow' method.")
        return {"error": "Missing run_full_workflow"}

    # Prepare inputs
    kwargs = {
        "resume": SAMPLE_RESUME_BEFORE,
        "target_position": "AI Engineer",
        "job_description": SAMPLE_JOB,
        "company_name": "Capgemini Engineering",
    }

    # Execute workflow
    try:
        if inspect.iscoroutinefunction(workflow_fn):
            results = await asyncio.wait_for(workflow_fn(**kwargs), timeout=300)  # 5 min timeout
        else:
            loop = asyncio.get_running_loop()
            results = await loop.run_in_executor(None, lambda: workflow_fn(**kwargs))
    except asyncio.TimeoutError:
        print("ERROR: Workflow timed out after 5 minutes")
        return {"error": "Workflow timeout"}
    except Exception as e:
        print(f"ERROR running workflow: {e}")
        traceback.print_exc()
        return {"error": "Workflow execution failed", "trace": traceback.format_exc()}

    # Display results
    print("\n" + "-"*80)
    print("WORKFLOW COMPLETE")
    print("-"*80)

    if not results:
        print("No results returned.")
        return results

    for key, value in results.items():
        print(f"\n{'='*80}")
        print(f"{key.upper().replace('_', ' ')}")
        print("="*80)

        if isinstance(value, dict):
            for k, v in value.items():
                s = str(v)
                print(f"  • {k}: {s[:300]}{'...' if len(s) > 300 else ''}")
        elif isinstance(value, (list, tuple)):
            for i, item in enumerate(value):
                s = str(item)
                print(f"  [{i+1}] {s[:300]}{'...' if len(s) > 300 else ''}")
        else:
            s = str(value)
            max_len = 1000 if "error" not in key.lower() else 500
            print(s[:max_len] + ("..." if len(s) > max_len else ""))

    # Summary statistics
    print("\n" + "="*80)
    print("SUMMARY")
    print("="*80)
    success_count = sum(1 for k in results.keys() if "error" not in k)
    error_count = sum(1 for k in results.keys() if "error" in k)
    print(f"Successful steps: {success_count}")
    print(f"Failed steps: {error_count}")

    return results

result = await run_demo()

CAREERFORGE AI - COMPLETE DEMO
✅ CareerForge AI System initialized with 5 agents

🔍 Step 1/5: Searching for jobs...
SerpAPI top keys: ['search_metadata', 'search_parameters', 'filters', 'jobs_results', 'serpapi_pagination']
SerpAPI returned 10 job(s) after parsing.

 Initial fetch: 10 jobs
  Entry Level AI Software Engineer                   Score: 55/100
  AI Software Engineer (junior career, hybrid, Secre Score: 65/100
  Robotics  AI Engineer  Entry Level                 Score: 62/100
  Data-Driven AI/ML Technology Solutions Engineer (E Score: 55/100
  Entry Level Hire  Remote Technical Support - AI an Score: 50/100
  AI-Driven .NET Software Engineer — Entry Level     Score: 60/100
  AI Engineer - Entry Level, Part-Time               Score: 64/100
  Associate AI Engineer                              Score: 60/100
  Innovation Intern - AI Engineering                 Score: 60/100
  Entry to Mid-Level Agentic AI Engineer             Score: 60/100

FINAL RESULTS: 5 jobs
1. AI Software E

In [66]:
# Output Formatter

import json
import os
import re
from IPython.display import Markdown, display, HTML
from datetime import datetime

OUT_DIR = "/mnt/data/careerforge_outputs"
os.makedirs(OUT_DIR, exist_ok=True)

# HELPER FUNCTIONS

def _try_parse(s):
    """Try to parse JSON from string or return as-is"""
    if isinstance(s, (dict, list)):
        return s
    if not isinstance(s, str):
        return None
    s = s.strip()

    # JSON parse
    try:
        return json.loads(s)
    except Exception:
        pass

    # Strip markdown code blocks
    patterns = [
        r"```(?:json)?\s*(\{.*\}|\[.*\])\s*```",
        r"```(?:json)?\s*(.*?)\s*```",
    ]

    for pattern in patterns:
        m = re.search(pattern, s, flags=re.DOTALL)
        if m:
            try:
                return json.loads(m.group(1))
            except Exception:
                pass

    return None


def _save_file(key, obj, raw):
    """Save parsed JSON or raw text to file"""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

    if obj is not None:
        path = os.path.join(OUT_DIR, f"{key}_{timestamp}.json")
        with open(path, "w", encoding="utf-8") as f:
            json.dump(obj, f, indent=2, ensure_ascii=False)
    else:
        path = os.path.join(OUT_DIR, f"{key}_{timestamp}.txt")
        with open(path, "w", encoding="utf-8") as f:
            f.write(raw or "")

    return path


def _print_section_header(title, emoji=""):
    """Print a beautiful section header"""
    print("\n" + "=" * 160)
    print(f"{emoji} {title}".center(80) if emoji else title.center(80))
    print("=" * 160 + "\n")


def _print_subsection(title):
    """Print a subsection divider"""
    print("\n" + "-" * 160)
    print(f"  {title}")
    print("-" * 160 + "\n")


def _truncate(text, max_length=500, add_ellipsis=True):
    """Truncate text intelligently"""
    if not text or len(str(text)) <= max_length:
        return str(text)

    truncated = str(text)[:max_length].rsplit(' ', 1)[0]
    return truncated + "..." if add_ellipsis else truncated

# SECTION FORMATTERS

def format_job_search(result):
    """Format job search results"""
    _print_section_header("JOB SEARCH RESULTS", "🔍")

    jobs_raw = result.get("job_search") or result.get("job_search_error")
    jobs = _try_parse(jobs_raw)

    # Handle nested structure
    if isinstance(jobs, dict):
        if "job_matches" in jobs:
            jobs = jobs["job_matches"]
        elif "top_matches" in jobs:
            jobs = jobs["top_matches"]
        elif "jobs" in jobs:
            jobs = jobs["jobs"]

    if isinstance(jobs, list) and len(jobs) > 0:
        print(f"📊 Found {len(jobs)} matching positions\n")

        for i, job in enumerate(jobs, 1):
            title = job.get("title") or job.get("job_title") or "Unknown Position"
            company = job.get("company") or "Unknown Company"
            location = job.get("location") or "Location not specified"
            posted = job.get("posted_time") or job.get("posted_hours_ago") or "Recently"
            salary = job.get("salary_range") or job.get("salary") or "Not specified"
            score = job.get("match_score", "")
            reason = job.get("match_reason") or job.get("reason") or ""
            url = job.get("url") or ""

            print(f"┌─ Job #{i} {'─' * 160}")
            print(f"│ 📌 {title}")
            print(f"│ 🏢 {company}")
            print(f"│ 📍 {location}")
            print(f"│ 💰 Salary: {salary}")
            print(f"│ ⏰ Posted: {posted}")

            if score:
                print(f"│ ⭐ Match Score: {score}/100")

            if reason:
                print(f"│ ✅ Why it matches:")
                # Word wrap reason
                for line in _wrap_text(reason, width=70):
                    print(f"│    {line}")

            if url:
                print(f"│ 🔗 Apply: {url}")

            print(f"└{'─' * 160}\n")

    else:
        print("ℹ️  No jobs found or parsing failed")
        if isinstance(jobs_raw, str):
            print(f"\nRaw output:\n{jobs_raw[:500]}...")

    # Save to file
    path = _save_file("job_search", jobs, str(jobs_raw))
    print(f"💾 Saved to: {path}")


def format_tailored_resume(result):
    """Format tailored resume"""
    _print_section_header("TAILORED RESUME", "📝")

    tailored_raw = result.get("tailored_resume") or result.get("tailored_resume_error") or ""
    tailored = _try_parse(tailored_raw)

    if isinstance(tailored, dict):
        # Handle structured resume
        suggested = tailored.get("suggested_changes") or []
        optimized = tailored.get("optimized_resume") or tailored.get("resume") or ""

        if suggested:
            print("🎯 Suggested Changes:\n")
            for i, change in enumerate(suggested, 1):
                print(f"{i}. {change}\n")

        if optimized:
            print("\n📄 Optimized Resume:\n")
            print(optimized)

    elif isinstance(tailored_raw, str) and len(tailored_raw) > 100:
        # Markdown resume
        print("📄 Full Resume (Markdown):\n")
        display(Markdown(tailored_raw))

    else:
        print("ℹ️  No resume data or parsing failed")
        print(f"\nRaw output:\n{str(tailored_raw)[:500]}...")

    # Save to file
    path = _save_file("tailored_resume", tailored, str(tailored_raw))
    print(f"\n💾 Saved to: {path}")


def format_ats_analysis(result):
    """Format ATS analysis"""
    _print_section_header("ATS COMPATIBILITY ANALYSIS", "🎯")

    ats_raw = result.get("ats_analysis") or result.get("ats_analysis_error")
    ats = _try_parse(ats_raw)

    if isinstance(ats, dict):
        # Overall score
        overall = ats.get("overall_score") or ats.get("overall") or ats.get("score")
        if overall:
            score_val = float(overall)
            emoji = "🟢" if score_val >= 80 else "🟡" if score_val >= 60 else "🔴"
            print(f"{emoji} Overall ATS Score: {overall}/100\n")

        # Score breakdown
        breakdown = ats.get("breakdown") or {}
        if breakdown:
            print("📊 Score Breakdown:\n")
            for category, score in breakdown.items():
                bar = "█" * int(float(score) / 5) + "░" * (20 - int(float(score) / 5))
                print(f"  {category:20s} [{bar}] {score}/100")
            print()

        # Missing skills
        missing = ats.get("missing_skills") or ats.get("missing") or []
        if missing:
            print(f"⚠️  Missing Keywords ({len(missing)} total):\n")
            for i in range(0, len(missing[:20]), 4):
                row = missing[i:i+4]
                print("  " + " | ".join(f"{skill:18s}" for skill in row))
            if len(missing) > 20:
                print(f"\n  ... and {len(missing) - 20} more")
            print()

        # Recommendations
        recs = ats.get("recommended_changes") or ats.get("suggestions") or []
        if recs:
            print(f"💡 Recommendations ({len(recs)} changes):\n")
            for i, rec in enumerate(recs, 1):
                if isinstance(rec, str):
                    text = rec
                elif isinstance(rec, dict):
                    text = rec.get("text") or rec.get("change") or rec.get("suggestion") or json.dumps(rec)
                else:
                    text = str(rec)

                # Word wrap
                print(f"{i}. ", end="")
                for line in _wrap_text(text, width=75):
                    print(f"   {line}")
                print()

    else:
        print("ℹ️  No ATS data or parsing failed")
        print(f"\nRaw output:\n{str(ats_raw)[:500]}...")

    # Save to file
    path = _save_file("ats_analysis", ats, str(ats_raw))
    print(f"💾 Saved to: {path}")


def format_interview_prep(result):
    """Format interview prep materials"""
    _print_section_header("INTERVIEW PREPARATION", "💼")

    ip_raw = result.get("interview_prep") or result.get("interview_prep_error")
    ip = _try_parse(ip_raw)

    if isinstance(ip, dict):
        # Technical questions
        tqs = ip.get("technical_questions") or ip.get("technical") or []
        if tqs:
            _print_subsection("🔧 Technical Questions")

            for i, q in enumerate(tqs, 1):
                print(f"\n{'─' * 160}")
                print(f"Question {i}/{len(tqs)}")
                print('─' * 160)

                if isinstance(q, dict):
                    question = q.get("question") or ""
                    why = q.get("why_asked") or q.get("why") or ""
                    key_points = q.get("key_points") or q.get("key_points_to_cover") or ""
                    example = q.get("example_answer") or q.get("example") or ""

                    print(f"\n❓ {question}\n")

                    if why:
                        print(f"📌 Why they ask this:")
                        for line in _wrap_text(why, width=70):
                            print(f"   {line}")
                        print()

                    if key_points:
                        print(f"🎯 Key points to cover:")
                        if isinstance(key_points, list):
                            for point in key_points:
                                print(f"   • {point}")
                        else:
                            for line in str(key_points).split("\n"):
                                if line.strip():
                                    print(f"   • {line.strip()}")
                        print()

                    if example:
                        print(f"💬 Example answer:")
                        for line in _wrap_text(example, width=70):
                            print(f"   {line}")
                        print()
                else:
                    print(f"\n❓ {str(q)}\n")

        # Behavioral questions
        bqs = ip.get("behavioral_questions") or ip.get("behavioral") or []
        if bqs:
            _print_subsection("🤝 Behavioral Questions (STAR Method)")

            for i, q in enumerate(bqs, 1):
                print(f"\n{'─' * 160}")
                print(f"Question {i}/{len(bqs)}")
                print('─' * 160)

                if isinstance(q, dict):
                    question = q.get("question") or ""
                    tips = q.get("key_points") or q.get("tips") or ""
                    example = q.get("example_answer") or q.get("example") or ""

                    print(f"\n❓ {question}\n")

                    if tips:
                        print(f"💡 Tips:")
                        for line in _wrap_text(tips, width=70):
                            print(f"   {line}")
                        print()

                    if example:
                        print(f"💬 Example (STAR format):")
                        for line in _wrap_text(example, width=70):
                            print(f"   {line}")
                        print()
                else:
                    print(f"\n❓ {str(q)}\n")

        # Company insights
        insights = ip.get("company_insights") or []
        if insights:
            _print_subsection("🏢 Company Insights")
            if isinstance(insights, str):
                print(f"• {insights}")
            elif isinstance(insights, list):
                for insight in insights:
                    if isinstance(insight, str):
                        print(f"• {insight}")
                    else:
                        print(f"• {str(insight)}")
            print()

        # Talking points
        talking_points = ip.get("talking_points") or []
        if talking_points:
            _print_subsection("🗣️ Key Talking Points")
            if isinstance(talking_points, str):
                print(f"✓ {talking_points}")
            elif isinstance(talking_points, list):
                for point in talking_points:
                    if isinstance(point, str):
                        print(f"✓ {point}")
                    else:
                        print(f"✓ {str(point)}")
            print()

    else:
        print("ℹ️  No interview prep data or parsing failed")
        print(f"\nRaw output:\n{str(ip_raw)[:500]}...")

    # Save to file
    path = _save_file("interview_prep", ip, str(ip_raw))
    print(f"💾 Saved to: {path}")


def format_salary_strategy(result):
    """Format salary negotiation strategy"""
    _print_section_header("SALARY NEGOTIATION STRATEGY", "💰")

    sal_raw = (
        result.get("salary_strategy") or
        result.get("salary_strategy_error") or
        result.get("negotiation_strategy")
    )
    sal = _try_parse(sal_raw)

    # Handle nested structure
    if isinstance(sal, dict) and "negotiation_strategy" in sal:
        analysis = sal.get("analysis", {})
        sal = sal["negotiation_strategy"]
    else:
        analysis = {}

    if isinstance(sal, dict):
        # Market data
        target_range = sal.get("target_range") or analysis.get("market_range")
        market_median = analysis.get("market_median")

        print("📊 Market Analysis:\n")
        if market_median:
            print(f"   Market Median: ${market_median:,}")
        if target_range:
            if isinstance(target_range, str):
                print(f"   Target Range:  {target_range}")
            else:
                print(f"   Target Range:  ${target_range[0]:,} - ${target_range[1]:,}")
        print()

        # Negotiation numbers
        print("💵 Negotiation Strategy:\n")
        counter = sal.get("counter_offer")
        fallback = sal.get("fallback_offer") or sal.get("fallback")
        walk_away = sal.get("walk_away_point")
        confidence = sal.get("confidence")

        if counter:
            print(f"   🎯 Counter Offer:   ${counter:,}")
        if fallback:
            print(f"   ⚖️  Fallback Offer:  ${fallback:,}")
        if walk_away:
            print(f"   🚪 Walk-Away Point: ${walk_away:,}")
        if confidence:
            print(f"   📈 Confidence:      {int(confidence * 100)}%")
        print()

        # Career transition info
        if analysis.get("is_career_transition"):
            print("⚠️  Career Transition Detected:\n")
            exp_breakdown = result.get("experience_breakdown", {})
            print(f"   Total Experience:    {exp_breakdown.get('total_years', 0)} years")
            print(f"   Relevant Experience: {exp_breakdown.get('relevant_years', 0)} years")
            print(f"   Level: {analysis.get('experience_level', 'Entry').title()}")
            print()

        # Conversation scripts
        scripts = sal.get("conversation_scripts") or sal.get("scripts") or []
        if scripts:
            print("💬 Conversation Scripts:\n")
            for i, script in enumerate(scripts, 1):
                print(f"{i}. ", end="")
                for line in _wrap_text(script, width=73):
                    print(f"   {line}")
                print()

        # Reasoning
        reasoning = sal.get("reasoning") or sal.get("explanation") or analysis.get("reasoning")
        if reasoning:
            print("📝 Strategy Reasoning:\n")
            for line in _wrap_text(reasoning, width=75):
                print(f"   {line}")
            print()

        # Data sources
        sources = analysis.get("sources_used", [])
        if sources:
            print("📚 Data Sources:\n")
            for source in sources:
                print(f"   • {source}")
            print()

    else:
        print("ℹ️  No salary data or parsing failed")
        print(f"\nRaw output:\n{str(sal_raw)[:500]}...")

    # Save to file
    path = _save_file("salary_strategy", sal, str(sal_raw))
    print(f"💾 Saved to: {path}")


def _wrap_text(text, width=70):
    """Word wrap text to specified width"""
    words = str(text).split()
    lines = []
    current_line = []
    current_length = 0

    for word in words:
        if current_length + len(word) + 1 <= width:
            current_line.append(word)
            current_length += len(word) + 1
        else:
            if current_line:
                lines.append(" ".join(current_line))
            current_line = [word]
            current_length = len(word)

    if current_line:
        lines.append(" ".join(current_line))

    return lines


def pretty_print_all(result):
    """
    Main function to beautifully print all CareerForge results

    Usage:
        pretty_print_all(result)
    """

    # Header
    print("\n" + "=" * 160)
    print("🚀 CAREERFORGE AI - COMPLETE ANALYSIS REPORT".center(80))
    print(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}".center(80))
    print("=" * 160)

    # Print each section
    format_job_search(result)
    format_tailored_resume(result)
    format_ats_analysis(result)
    format_interview_prep(result)
    format_salary_strategy(result)


pretty_print_all(result)


                  🚀 CAREERFORGE AI - COMPLETE ANALYSIS REPORT                   
                         Generated: 2025-11-29 01:49:41                         

                              🔍 JOB SEARCH RESULTS                              

📊 Found 5 matching positions

┌─ Job #1 ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
│ 📌 AI Software Developer- Healthcare Domain- Visa Independent
│ 🏢 Shrive Technologies
│ 📍 United States
│ 💰 Salary: Not specified
│ ⏰ Posted: 1 day ago
│ ⭐ Match Score: 74/100
│ ✅ Why it matches:
│    Strong match • skills: Python, Go • Very recent posting
│ 🔗 Apply: https://www.linkedin.com/jobs/view/ai-software-developer-healthcare-domain-visa-independent-at-shrive-technologies-4339257143?utm_campaign=google_jobs_apply&utm_source=google_jobs_apply&utm_medium=organic
└──────────────────────────────────────────────────────────────────────────────